# Jupiter Notebook to test csv_stuff.py

In [1]:
import pandas as pd
import sys
import re
import random
from bs4 import BeautifulSoup
import argparse
import matplotlib.pyplot as plt
import difflib
from csv_stuff import *

In [ ]:
semester='SoSe21'
ha='9'
# tasks = ['Antwort 9']
tasks = ['Antwort 8', 'Antwort 9', 'Antwort 10']
prog_language='C'
df_labled, count = create_labled_table_routine(semester=semester, ha=ha, tasks=tasks, prog_language=prog_language)
# answerpreload = get_given_code(f'../../data/code_templates/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_{tasks[0]}.xml')

### Small calculation to get number of all possible pairs in PPR

In [46]:
semester='SoSe21'
ha=['6','7','8','9','10']   #5 relevant programm homework weeks + (possible exam tasks)
abgaben=[34,34,34,34,34]    #SoSe2021 34 student only in c
tasks = [4, 3, 3, 3, 3]     #2-4 programm tasks per week
summe=0
for abgabe,task in zip(abgaben,tasks):
    summe += (abgabe-1)**2*task     #with 34 comes 33 pairs per student per task per week
print(summe)                #number of homework pairs in one semester
print(summe*4)              #number of pairs in four semester
print(summe*4*2)            #number of pairs in four semester in 2 programming languages


17424
69696
139392


## Test difflib with hand-labled data

In [24]:
import difflib
import pandas as pd
import sys
from os import listdir
from os.path import isfile, join
import math

def difflib_on_directory(path):
    file_names = [f for f in listdir(path) if isfile(join(path, f))]
    if len(file_names) < 2:
        print('no files to compare')
    file_contents = [open(f).readlines() for f in file_names]
    num_files = len(file_names)
    sims = []
    for x in range(0, num_files):
        for y in range(x+1, num_files):
            s = difflib.SequenceMatcher(None, file_contents[x], file_contents[y])
            ratio = s.ratio()
            sims.append((file_names[x], file_names[y], ratio))

    sorted_sims = sorted(sims, key=lambda s: -s[2])

    for f1, f2, score in sorted_sims:
        print(f1 + ',' + f2 + ',' + str(score))

def difflib_on_csv(path):
    df = pd.read_csv(path, delimiter=',')
    sims = []
    for sn, code1, code2 in df[['surname1', 'code1', 'code2']].values:
        s = difflib.SequenceMatcher(None, code1, code2)
        ratio = s.ratio()
        # print((sn, ratio))
        sims.append((sn, ratio))

    sorted_sims = sorted(sims, key=lambda s: -s[1])

    for sn, score in sorted_sims[0:5]:
        print(sn + ',' + sn + ',' + str(score))
    for sn, score in sorted_sims[-6:-1]:
        # print('HUHU')
        print(sn + ',' + sn + ',' + str(score))

difflib_on_csv("/home/erik/TU/ni/plagiate_labeltool/data/labled/PPR [SoSe21]-9. Hausaufgabe - Pflichttest C-Antworten_plagiate.csv")


Yang Felix,Yang Felix,0.7399741267787839
Yang Felix,Yang Felix,0.7382492453643812
Yang Felix,Yang Felix,0.728762397585166
Yang Felix,Yang Felix,0.7278999568779646
Yang Felix,Yang Felix,0.7253126347563605
Tom Alexander,Tom Alexander,0.3467550923732828
Maximilian Erich,Maximilian Erich,0.3433962264150943
Maximilian Erich,Maximilian Erich,0.31761006289308175
Jared Louis,Jared Louis,0.2912216723104975
Tom Alexander,Tom Alexander,0.2273803884414969


In [3]:
# creates c files out of csv's
df = pd.read_csv('~/TU/ni/plagiate_labeltool/data/labled/PPR [SoSe21]-7. Hausaufgabe - Pflichttest C-Antworten_labled.csv', delimiter=',')
sum = 0
for row in df[['lastname1','code1']].values:
    if sum == 15:
        # print((row[0], row[1]))
        sum = 0
        with open(f"{row[0]}.c", "w") as file:
            file.write(row[1])
    sum = sum + 1


## Create auto-labeld data with difflib

In [1]:
import pandas as pd
csv_path = "/home/erik/TU/ni/plagiate_labeltool/data/labled/PPR [SoSe22]-9. Hausaufgabe - Pflichttest C-Antworten_labled.csv"
csv_path2 = "/home/erik/TU/ni/plagiate_labeltool/data/labled/PPR [SoSe22]-9. Hausaufgabe - Pflichttest C-Antworten_autolabled.csv"
df = pd.read_csv(csv_path, delimiter=',')
# df.to_csv(csv_path)


In [ ]:
import pandas as pd
import difflib

def label_csv_with_difflib(csv_path):
    try:
        df = pd.read_csv(csv_path, delimiter=',')
        for i, row in enumerate(df[['code1', 'code2']].values):
            s = difflib.SequenceMatcher(None, row[0], row[1])
            df.loc[i, ['label']] = s.ratio()
           
    except:
        return f"Couldnt open {csv_path}"


In [ ]:
for row in df.nlargest(20, columns=['label'])[['Unnamed: 0', 'surname1', 'surname2', 'label', 'difflib_']].values:
    print(f"{row[1]}, {row[2]}, {row[3]}, {row[4]}")


In [ ]:
import numpy as np

for row in df.nlargest(20, columns=['label'])[['Unnamed: 0', 'surname1', 'surname2', 'label']].values:
    print(f"{row[1]}, {row[2]}, {row[3]}, {np.abs(df.loc[(df['surname1']==row[2]) & (df['surname2']==row[1])]['label'].values[0] - row[3])}")


## Create auto-labeld data with Levenshtein

In [12]:
from Levenshtein import distance
from sklearn.manifold import MDS

def distance_percentage(string1, string2):
    return distance(string1, string2) / np.max([len(string1), len(string2)])


def get_similarity(string1, string2):


    distance1 = distance_percentage(string1, string2)

    string1_p = string1
    string1 = string1.split()
    string1_w = "".join(string1)
    string1_s = "".join(sorted(string1_w))

    string2_p = string2
    string2 = string2.split()
    string2_w = "".join(string2)
    string2_s = "".join(sorted(string2_w))

    distance2 = distance_percentage(string1_w, string2_w)
    distance3 = distance_percentage(string1_s, string2_s)


    return 1 - distance2#, 1 - distance3


In [13]:
for i, row in enumerate(df[['code1', 'code2']].values):
    label = get_similarity(row[0], row[1])
    # if label >= 0.6:
    #     label = np.round(label)
    # elif label <= 0.35:
    #     label = np.floor(label)
    df.loc[i, ['Levenshtein2']] = label
# df.to_csv(csv_path2)


In [ ]:
df_labled = pd.DataFrame(columns=['semester', 'ha', 'task', 'prog_lang',
                            'surname1', 'lastname1', 'surname2', 'lastname2', 'code1', 'code2', 'label'])
df_labled.index.name = 'index'
for task in tasks:
    for i, ln1, sn1, code1 in enumerate(df.loc[df[f'{task} empty'] == 0][['Nachname', 'Vorname', task]].values):
        for ln2, sn2, code2 in df.loc[(df[f'{task} empty'] == 0) & (df[task] != code1) & ()][['Nachname', 'Vorname', task]][i:].values:
            df_labled.loc[len(df_labled)] = [
                semester, ha, task, prog_language, sn1, ln1, sn2, ln2, code1, code2, -1]


In [19]:
for i,j in enumerate(range(0,3)):
    print((i,j))
df[0:2]

(0, 0)
(1, 1)
(2, 2)


,Unnamed: 0,semester,ha,task,prog_lang,surname1,lastname1,surname2,lastname2,code1,code2,label,Levenshtein,Levenshtein2
0,0,SoSe22,9,Antwort 9,C,Fabian,Große-Wöhrmann,Moritz Sebastian,Niethammer,#include <stdio.h>\r\n#include <stdlib.h>\r\n ...,#include <stdio.h>\r\n#include <stdlib.h>\r\n\...,0.521108,0.673325,0.761993
1,1,SoSe22,9,Antwort 9,C,Fabian,Große-Wöhrmann,Florian,Schacht,#include <stdio.h>\r\n#include <stdlib.h>\r\n ...,//gcc -x c HA09_F09.c -o HA09_F09 -Wall -Wextr...,0.051832,0.157407,0.156506


## Postprocess csv file

### Delete rows consist of a pair with answerpreload from another task

In [39]:
import pandas as pd
csv_path = f'../../data/labled/PPR [WS2021]-9. Hausaufgabe - Pflichttest C-Antworten_labled.csv'
df = pd.read_csv(csv_path, delimiter=',')
df = df.drop('Unnamed: 0', axis=1)
df.head(4)


,semester,ha,task,prog_lang,surname1,lastname1,surname2,lastname2,code1,code2,label,hand_labled
0,WS2021,9,Antwort 10,C,Jolanda Marietta,Schumann,David,Stubler,#include <stdio.h>\r\n\r\nvoid aussortieren (c...,#include <stdio.h>\r\n#include <stdlib.h>\r\n\...,0.0,1
1,WS2021,9,Antwort 10,C,Jolanda Marietta,Schumann,Daniel Dominik,Eichstädt,#include <stdio.h>\r\n\r\nvoid aussortieren (c...,#include <stdio.h>\r\n\r\nchar aussortieren (c...,0.0,1
2,WS2021,9,Antwort 10,C,Jolanda Marietta,Schumann,Mohamad Anas,Allaham,#include <stdio.h>\r\n\r\nvoid aussortieren (c...,#include <stdio.h>\r\n#include <stdlib.h>\r\n\...,0.0,1
3,WS2021,9,Antwort 10,C,Jolanda Marietta,Schumann,Moritz,Wahl,#include <stdio.h>\r\n\r\nvoid aussortieren (c...,#include <stdio.h>\r\n#include <stdlib.h>\r\n ...,0.0,1


In [ ]:
semester='WS2021'
ha='9'
prog_language='C'
task='Antwort 9'
answerpreload, _ = get_given_code(
            f'../../data/code_templates/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_{task}.xml')
print(answerpreload)

In [34]:

def compare_with_given_code(student_code, semester, ha, task, prog_language):
    # answerpreload = get_given_code(f'../../data/code_templates/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_{task}.xml')
    answerpreload, _ = get_given_code(
        f'../../data/code_templates/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_{task}.xml')
    answerpreload = answerpreload.replace(
        '\t', '').replace('\r', '').replace('\n', '')
    # every special character beetween \Q and \E is ignored, but the inserted \S*; answers shouldnt contain \Q and \E!
    answerpreload = re.escape(answerpreload)
    # {{ cr_random.f1 }} --> \S*
    answerpreload = re.sub(
        r"\\{\\{\\\s*\S+\s*\\}\\}", r"\\S*", answerpreload)
    # there are \\n in the student solutions because "...\n" --> "...\\n" while read_csv
    student_code = student_code.replace('\t', '').replace(
        '\r', '').replace('\n', '').replace('\\n', r'\n')
    # test if template was given or not
    if answerpreload != '':
        result = re.match(answerpreload, student_code)
    return result != None


In [62]:
# summe=0
# for i, (code1, code2) in enumerate(df[['code1','code2']].values):
#     compare1 = compare_with_given_code(code1,ha=ha,prog_language=prog_language,semester=semester,task=task)
#     compare2 = compare_with_given_code(code2,ha=ha,prog_language=prog_language,semester=semester,task=task)
#     # print((i, lastname1, lastname2, compare1, compare2))
#     if compare1 or compare2:
#         df = df.drop(i)
#         summe+=1
# print(summe)
df.to_csv(f'/home/erik/TU/ni/plagiate_labeltool/data/labled/PPR [WS2021]-9. Hausaufgabe - Pflichttest C-Antworten_labled_2.csv')


## Obfuscating the Student Source Code to create plagiarism pairs

In [24]:
from obfuscator import comment_remover, variable_renamer
import pandas as pd

def create_plagiate_table(df, semester, ha, task, prog_language, number_labled_pairs, path):
    df_labled = pd.DataFrame(columns=['semester', 'ha', 'task', 'prog_lang',
                                    'surname1', 'lastname1', 'code1', 'code2', 'label'])

    df = add_valid_code_columns(df, semester, ha, [task], prog_language)
    number_of_valid_codes = len(df.loc[df[f'{task} empty'] == 0])
    print(number_of_valid_codes)
    plagiate_per_solution = np.floor(number_labled_pairs/number_of_valid_codes)
    print(plagiate_per_solution)
    # for ln1, sn1, code1 in df.loc[df[f'{task} empty'] == 0][['Nachname', 'Vorname', task]].values:
    for ln, sn, code in df.loc[df[f'{task} empty'] == 0][['Nachname', 'Vorname', task]].values:
        for i in range(int(plagiate_per_solution)):
            plagiate_code = comment_remover(code)
            plagiate_code = variable_renamer(plagiate_code)
            df_labled.loc[len(df_labled)] = [
                semester, ha, task, prog_language, sn, ln, code, plagiate_code, 1]
    df_labled.to_csv(path)
    return df_labled

def create_plagiate_table_routine(df, semester, ha, task, prog_language):
    csv_path = f'../../data/raw_data/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten.csv'
    csv_path1 = f'../../data/labled/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_labled.csv'
    csv_path2 = f'../../data/labled/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_plagiate.csv'
    df = pd.read_csv(csv_path, delimiter=',')
    number_labled_pairs = len(pd.read_csv(csv_path1, delimiter=','))
    df_labled = create_plagiate_table(df, semester, ha, task, prog_language, number_labled_pairs, csv_path2)
    return df_labled

semester='SoSe21'
ha='9'
prog_language='C'
task='Antwort 9'
df_labled = create_plagiate_table_routine(df, semester, ha, task, prog_language)



18
8.0


In [17]:
import numpy as np
from obfuscator import comment_remover, variable_renamer

df_labled = pd.DataFrame(columns=['semester', 'ha', 'task', 'prog_lang',
                                  'surname1', 'lastname1', 'code1', 'code2', 'label'])

number_of_valid_codes = len(df.loc[df[f'{task} empty'] == 0])
print(number_of_valid_codes)
df = add_valid_code_columns(df, semester, ha, [task], prog_language)
plagiate_per_solution = np.floor(number_labled_pairs/number_of_valid_codes)
print(plagiate_per_solution)
# for ln1, sn1, code1 in df.loc[df[f'{task} empty'] == 0][['Nachname', 'Vorname', task]].values:
for ln, sn, code in df.loc[df[f'{task} empty'] == 0][['Nachname', 'Vorname', task]].values:
    for i in range(int(plagiate_per_solution)):
        plagiate_code = comment_remover(code)
        plagiate_code = variable_renamer(plagiate_code)
        df_labled.loc[len(df_labled)] = [
            semester, ha, task, prog_language, sn, ln, code, plagiate_code, 1]


18
8.0
